In [3]:
import pandas as pd

train_df = pd.read_csv("/content/train.csv")
print(train_df.head())

         ImageId  ClassId
0  0002cc93b.jpg      1.0
1  0007a71bf.jpg      3.0
2  000a4bcdd.jpg      1.0
3  000f6bf48.jpg      4.0
4  0014fce06.jpg      3.0


In [4]:
dominant_classes = train_df.groupby('ImageId')['ClassId'].agg(lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else -1)



In [5]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])


In [6]:
from torch.utils.data import Dataset
from PIL import Image
import os

class SteelDataset(Dataset):
    def __init__(self, df, image_folder, transform=None):
        self.df = df
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['ImageId']
        label = self.df.iloc[idx]['ClassId'] - 1
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label



In [7]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


In [8]:
from torch.utils.data import DataLoader

train_dataset = SteelDataset(train_df, "../data/train_images/", transform=transform)
val_dataset = SteelDataset(val_df, "../data/train_images/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 151MB/s]


In [19]:
import pandas as pd
import os
from PIL import Image


test_image_folder = "/content/"
test_image_name = "3036d3834.jpg"

predictions = []


img_path = os.path.join(test_image_folder, test_image_name)
image = Image.open(img_path).convert("RGB")
image = transform(image).unsqueeze(0).to(device)


with torch.no_grad():
    model.eval()
    output = model(image)
    _, predicted = torch.max(output, 1)

predictions.append((test_image_name, predicted.item() + 1))

df = pd.DataFrame(predictions, columns=['ImageId', 'ClassId'])
df.to_csv("submission.csv", index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
